In [7]:
!source pyvis_venv/bin/activate

In [8]:
!pip3 uninstall pyvis

Found existing installation: pyvis 0.3.1
Uninstalling pyvis-0.3.1:
  Would remove:
    /Users/yash.chhabria/project-test/Control-plane-help/.venv/lib/python3.10/site-packages/pyvis-0.3.1.dist-info/*
    /Users/yash.chhabria/project-test/Control-plane-help/.venv/lib/python3.10/site-packages/pyvis/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [ ]:
!pip3 install networkx
!pip3 install pyvis==0.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.9/748.9 kB 6.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached jsonpickle-3.0.1-py2.py3-none-any.whl (40 kB)
  Using cached MarkupSafe-2.1.2-cp310-cp310-macosx_10_9_universal2.whl (17 kB)
  Created wheel for pyvis: filename=pyvis-0.3.1-py3-none-any.whl size=755834 sha256=130c03e75b68881d82bae4c16a391bf90109edcf8f678b3e8799c0adb5565dfa
  Stored in directory: /Users/yash.chhabria/Library/Caches/pip/wheels/37/f9/93/44dd6cbfb2ead35307b114d27af8c6a14d5762a462af1e04f5
Successfully built pyvis


In [61]:
import json
import networkx as nx
import os

from pyvis.network import Network

def create_nodes_list(data):
    nodes_to_ingest =  []
    for node in data.keys():
        if data[node]['node_ip'] not in nodes_to_ingest: #for node_ip nodes
            nodes_to_ingest.append("node("+data[node]['node_ip']+')')
        node_with_attr_tuple = (node , data[node])
        nodes_to_ingest.append(node_with_attr_tuple)

    return nodes_to_ingest


def create_edges_list(data):
    edges_to_ingest = []
    for node_key, node_values in data.items():
        for connection in node_values['incomming_connections']:
            # if connection[0].startswith('10.2.152.169') or connection[1][0].startswith('10.2.152.169'):
            if connection[1][0].startswith('10.2.'):
                edge_attrs = {'sending' : connection[1][0].split(':')[1],
                            'listening' : connection[0].split(':')[1] 
                            }
                edge_tuple = (connection[1][1], node_key, edge_attrs)
                edges_to_ingest.append(edge_tuple)


            #for node to pod connections

            if connection[1][0].startswith('10.20.'):
                edge_attrs = {'sending' : connection[1][0].split(':')[1],
                            'listening' : connection[0].split(':')[1] 
                            }
                
                edge_tuple = ("node("+connection[1][0].split(":")[0]+')', node_key, edge_attrs)
                edges_to_ingest.append(edge_tuple)
                
                #print(connection[1][1], node_key, 'sending:', connection[1][0].split(':')[1], 'listening:', connection[0].split(':')[1])
    return edges_to_ingest




G = nx.DiGraph()
json_file = 'json_pod_parsed.json'
with open(json_file) as json_file:
    data = json.load(json_file)
nodes_to_ingest = create_nodes_list(data)
edges_to_ingest = create_edges_list(data)
G.add_nodes_from(nodes_to_ingest)
# print(edges_to_ingest)
G.add_edges_from(edges_to_ingest)
# print(G.edges())
# nx.draw(G, with_labels=True)
# plt.show()

for node in G.nodes():
    # Convert node attributes to compatible types
    attributes = G.nodes[node]
    for attr, value in attributes.items():
        if isinstance(value, (int, float)):
            attributes[attr] = str(value)
        elif isinstance(value, dict) or isinstance(value, list):
            attributes[attr] = json.dumps(value)

for edge in G.edges():
    # Convert edge attributes to compatible types
    start_node, end_node = edge
    attributes = G.edges[start_node, end_node]
    for attr, value in attributes.items():
        if isinstance(value, (int, float)):
            attributes[attr] = str(value)
        elif isinstance(value, dict) or isinstance(value, list):
            attributes[attr] = json.dumps(value)


In [62]:
isolated_nodes_and_pods = list(nx.isolates(G))

isolated_nodes = []
for node in isolated_nodes_and_pods:
    if node.startswith('node('):
        isolated_nodes.append(node)

G.remove_nodes_from(isolated_nodes)

In [69]:
nt = Network('3000', '3000', directed=True,  select_menu=True, filter_menu=True)
nt.from_nx(G)

#nt.show_buttons()
nt.show_buttons(filter_=['nodes'])
nt.show('consumable_vis_pods_and_nodes.html')



In [64]:
dir(nt)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_edge',
 'add_edges',
 'add_node',
 'add_nodes',
 'barnes_hut',
 'bgcolor',
 'cdn_resources',
 'conf',
 'directed',
 'dot_lang',
 'edges',
 'filter_menu',
 'font_color',
 'force_atlas_2based',
 'from_DOT',
 'from_nx',
 'generate_html',
 'get_adj_list',
 'get_edges',
 'get_network_data',
 'get_node',
 'get_nodes',
 'heading',
 'height',
 'hrepulsion',
 'html',
 'inherit_edge_colors',
 'neighborhood_highlight',
 'neighbors',
 'node_ids',
 'node_map',
 'nodes',
 'num_edges',
 'num_nodes',
 'options',
 'path',
 'prep_notebook',
 'repulsion',
 'save_graph',
 'select_menu',
 'set_edge_smooth',
 'set_options',
 'set_template

In [54]:
pod_net_f = open("pod_networking.txt", "r")
pod_net_list = pod_net_f.readlines()
pod_net_f.close()

pod_ip_dict = {}

for i in range(0,len(pod_net_list)):
    try:
        if pod_net_list[i].startswith("****"):
            i += 1
            pod_info = pod_net_list[i].strip().split("|")

            #fix for chutiya service pod relationship file 
            if pod_info[1] not in pod_ip_dict.keys():
                pod_ip_dict[pod_info[0]] = pod_info[1]
    except:
        pass


In [55]:


priv_list = """sqlacl-sisyphus-c4fb4d997-cpmxz
Cons-webapp-0
cons-sisyphus-74ddbdc456-99g8v
cons-shs-8589bdf6c6-rddhj
cons-s3-commit-84dd85fcf6-bmxzx
Cons-manager-0
cons-jobs-runner-9766899db-9fdjh
Cons-jobs-0
cons-delegate-76586f55c-b6g62
auth-high-qps-6c786dd5b9-9wvj9
auth-c5944fbbf-cktr8
managed-catalog-b5685957-sk28b
cons-manager-0
cons-delegate-76586f55c-b6g62
Cluster-monitor-0
cluster-manager-k8s-57f4d497bc-k6m4c
cluster-libraries-89855b5cc-zjv9z
Cons-projects-0
directory-599fdd8684-dk7bz
strimzi-cluster-operator-5b55f5b6-qvmjl
"""
priv_list = priv_list.split()

In [56]:
pod_ip_dict.keys()

dict_keys(['access-control-78c6d875b8-rk98n', 'acl-manager-867cc96fcc-d2p49', 'apiproxy-68d6bc64fc-88tpk', 'auth-c5944fbbf-cktr8', 'auth-high-qps-6c786dd5b9-9wvj9', 'auth-v2-74df99ddbc-89jwg', 'automl-895b5bb78-pxlxn', 'billing-ingester-5584b47b6c-fbn2d', 'cloudmock-794bc6696b-h2lkv', 'cluster-events-5b4fdfb55d-z57mw', 'cluster-identity-7dc49d5fc4-fmq7f', 'cluster-libraries-89855b5cc-zjv9z', 'cluster-manager-k8s-57f4d497bc-k6m4c', 'cluster-metrics-6865bbf9f5-4z994', 'cluster-monitor-0', 'clusters-database-monitor-56c5f797c7-t5d22', 'cmapiserver-695cbbc467-zjddt', 'cons-delegate-76586f55c-b6g62', 'cons-jobs-0', 'cons-jobs-proxy-77dc498f9d-48v99', 'cons-jobs-runner-9766899db-9fdjh', 'cons-manager-0', 'cons-projects-0', 'cons-s3-commit-84dd85fcf6-bmxzx', 'cons-shs-8589bdf6c6-rddhj', 'cons-sisyphus-74ddbdc456-99g8v', 'cons-webapp-0', 'consul-0', 'credentials-manager-5b99dc45b5-dl6r6', 'data-monitoring-76d6b96c54-zw5ch', 'data-plane-router-578cfb7d9f-whs7z', 'delta-pipelines-74878b6ff5-qctq

In [57]:
priv_ips = []


for pod in priv_list:
    priv_ips.append(pod_ip_dict[pod.lower()])

In [58]:
for node in nt.nodes:
    if node['id'] in priv_list:
        node['color']='red'


In [59]:
for node in nt.nodes:
    if node['id'] in priv_list:
        print(node)

{'color': 'red', 'pod_ip': '10.2.152.169', 'node_ip': '10.20.10.162', 'listening_ports': '["5290", "6000", "443", "7777"]', 'outgoing_ports': '["5290", "58100", "52956", "44390", "45506", "36844", "57878", "51672", "44378", "58106", "36852"]', 'pid': '[["6", "com.databricks.auth.server.service.AuthService"]]', 'incomming_connections': '[["10.2.152.169:5290", ["10.2.152.173:56236", "lakeview-query-6f59dd4c7-484gq"], "ESTABLISHED", "6/java"], ["10.2.152.169:5290", ["10.2.50.106:54964", "credentials-manager-5b99dc45b5-dl6r6"], "ESTABLISHED", "6/java"], ["10.2.152.169:5290", ["10.2.211.121:56444", "cons-sisyphus-74ddbdc456-99g8v"], "ESTABLISHED", "6/java"], ["10.2.152.169:5290", ["10.2.98.205:46804", "sqlgateway-proxy-6d746ff9c9-9l6nq"], "ESTABLISHED", "6/java"], ["10.2.152.169:5290", ["10.2.12.188:54046", "sqlgateway-history-67d585ff87-gk7d7"], "ESTABLISHED", "6/java"], ["10.2.152.169:5290", ["10.2.50.107:59786", "user-activity-7c96cb868d-9gnkl"], "ESTABLISHED", "6/java"], ["10.2.152.169:

In [60]:
nt.show_buttons(filter_=['physics'])
nt.show('consumable_vis_pods_and_nodes.html')